<p style="font-weight:bold;"> <span style="font-size: 36px"> In Memory Initialization of unconfigured DataSource </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Reference Data </span> </p>

<a id='init-base-to-memory'></a>

In [0]:
#!import "../Constants/CalculationEngine"
#!eval calculationEngine

# Set default DataSource

In [0]:
await DataSource.SetAsync();
DataSource.Reset(x => x.ResetCurrentPartitions());

# Initialize Workspace

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
Workspace.InitializeFrom(DataSource);

# Import Dimensions

In [0]:
await Import.FromFile("../Files/Dimensions.csv")
                .WithType<Novelty>()
                .WithType<AocType>()
                .WithType<PnlVariableType>()
                .WithType<BsVariableType>()
                .WithType<AmountType>()
                .WithType<DeferrableAmountType>()
                .WithType<RiskDriver>()
                .WithType<EconomicBasis>()
                .WithType<EstimateType>()
                .WithType<ValuationApproach>()
                .WithType<LineOfBusiness>()
                .WithType<OciType>()
                .WithType<LiabilityType>()
                .WithType<Profitability>()
                .WithType<Currency>()
                .WithType<Partner>()
                .WithType<CreditRiskRating>()
                .WithType<Scenario>()
                .WithType<ProjectionConfiguration>()
                .WithTarget(DataSource)
                .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("../Files/Dimensions.csv")
                .WithFormat(ImportFormats.AocConfiguration)
                .WithTarget(DataSource)
                .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("../Files/ReportingNodes/ReportingNodes.csv")
                    .WithType<ReportingNode>()
                    .WithTarget(DataSource)
                    .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

# Import Parameters

In [0]:
var log = await Import.FromFile("../Files/Parameters/YieldCurve_2019_12.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader);
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_1.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)
        );
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_3.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)
        );
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_12.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)
        );
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_3.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)
        );
/*log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_6.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)
        );
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_12.csv")
                        .WithFormat(ImportFormats.YieldCurve)
                        .WithTarget(DataSource)
                        .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader));*/
log

In [0]:
await Import.FromFile("../Files/Parameters/ExchangeRate.csv")
                .WithType<ExchangeRate>()
                .WithTarget(DataSource)
                .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("../Files/Parameters/PartnerRating.csv")
                .WithType<PartnerRating>()
                .WithTarget(DataSource)
                .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("../Files/Parameters/CreditDefaultRate.csv")
                .WithType<CreditDefaultRate>()
                .WithTarget(DataSource)
                .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

# Reset Workspace

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());